## Extractor : Lobby Register 2024

In [3]:
from typing import Optional
import os
import sys
from pydantic import BaseModel, Field

import pandas as pd
import numpy as np

from langchain_mistralai import ChatMistralAI

In [4]:
# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import db_interaction
from utils import entity_extraction
from models import organisation_model

## Database ingestion

In [5]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker
import logging

# Initialize the base class for declarative class definitions
Base = declarative_base()

# Set up the engine and session
engine = create_engine('sqlite:///../db/organisations.db')
Base.metadata.create_all(engine)  # Create the table if it does not exist

Session = sessionmaker(bind=engine)
session = Session()

# Set up logging
logging.basicConfig(filename='../logs/insert_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

## Load datasets

In [7]:

path_full_data ="../datasets/inputs/organisations_in_transparency_register_jan_23.xlsx"
path_training_data = "../datasets/inputs/cleaned_network_data.xlsx" 

df_full = pd.read_excel(path_full_data,sheet_name='LIST_REGISTRED_ORGANISATION' )
df_training = pd.read_excel(path_training_data,sheet_name='training_solved')

In [8]:
df_full.head(2)

,Identification code,Registration date,Category of registration,Name,Acronym,Form of the entity,Website URL,Head office address,Head office post code,Head office post box,...,Source of funding (other),Closed year start,Closed year end,Closed year: Intermediary (cost) or client (revenue): EU legislative proposal,Closed year EU grant: amount (source),Closed year total EU grants,Current year Intermediary or client,Current year EU grant: source (amount),Current year total,Complementary information
0,875248845569-64,17/02/2022,Trade and business associations,ASOCIACIÓN DE ARMADORES DE CERCO DE GALICIA,ACERGA,ASOCIACIÓN SIN ÁNIMO DE LUCRO,http://acerga.com/,"AVDA. DEL PUERTO, 21-2º",15160,NaN,...,NaN,01/01/2021,01/12/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,805341845171-02,17/01/2022,Companies & groups,Sympower BV,NaN,BV,www.sympower.net,Prinsengracht 437A,1016HM,NaN,...,NaN,01/01/2020,01/12/2020,NaN,NaN,NaN,NaN,NaN,NaN,1. As the 2021 financial year has not yet been...


In [9]:
df_full_filtered = df_full[['Identification code','Is member of: List of associations, (con)federations, networks or other bodies of which the organisation is a member']]
df_full_filtered.rename(columns={'Is member of: List of associations, (con)federations, networks or other bodies of which the organisation is a member':'member_of',
                               'Identification code':'id' },
                        inplace=True)

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_36606/806738430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_filtered.rename(columns={'Is member of: List of associations, (con)federations, networks or other bodies of which the organisation is a member':'member_of',


### exclude empty / null rows

In [10]:
# Define a list of values to exclude
exclude_values = ["none.", "none", "keine", "aucun", "nessuna", "no"]

In [11]:
df_full_filtered = df_full_filtered.dropna(subset=['member_of'])
df_full_filtered = df_full_filtered[df_full_filtered['member_of'].notnull()]

df_full_filtered = df_full_filtered[~df_full_filtered['member_of'].str.lower().isin(exclude_values)]
len(df_full_filtered)

9793

## The Extractor

In [12]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Your task is to extract organisation names, URLs, and abbreviations. "
            "Abbreviations are words that are in all capital letters"
            "and they must be extracted as 'abbreviations' not 'organisation names'. "
            "Organisation names are typically not in all capital letters, and are not abbreviations. "
            "These value pairs are often delimited from other pairs by new lines, commas, dashes, or semicolons. "
            "If an organisation's abbreviation exists, always extract it separately. "
            "If a value is not present in the text, return null for that attribute.",
        ),
        #example_human,  # Example input from a human
        #example_assistant,
        ("user", "{text}"),
    ]
)

In [14]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the Mistral API key
mistral_api_key = os.getenv("MISTRAL_API_KEY")

llm

In [11]:
llm = ChatMistralAI(model="mistral-large-latest", temperature=0, api_key=mistral_api_key)

In [12]:
runnable = prompt | llm.with_structured_output(schema=organisation_model.Data)

## Exclude processed entires
1. Those ids with entries in the database exclude_processed_rows)
2. Those processed, but which did not contain any membership information (exclude_processed_rows_without_results)

In [13]:
df_full_filtered.head()

,id,member_of
0,875248845569-64,COMITÉ CONSULTIVO CC SUR - https://cc-sud.eu/...
1,805341845171-02,DR4EU: https://dr4eu.org/
2,513518246200-77,International Investment Funds Association \n...
4,1733114388-50,Eurelectric \nhttp://www.eurelectric.org/ \n \...
5,284400047930-54,Eurofer \nWorldsteel Association \nCzech Confe...


##### get information on database entires

In [14]:
results_all_entities = session.query(db_interaction.OrganisationModel).all()
results_organisation_count = session.query(db_interaction.OrganisationModel.custom_id).distinct().count()
print(f"Total records of organisations in database: {len(results_all_entities)} for {results_organisation_count} organisations")

# Verify inserted records
results_without = session.query(db_interaction.ProcessingLog).filter_by(status="no_info").all()
print(f"Total records of entires without results: {len(results_without)}")

Total records of organisations in database: 39062 for 9744 organisations
Total records of entires without results: 55


##### exclude processed information

In [19]:
# Apply process_data on each row
df = entity_extraction.exclude_processed_rows(df_full_filtered,session)
df = entity_extraction.exclude_processed_rows_without_results(df,session)

In [ ]:
print(f'there are {len(df)} organisations left for extracting information')

### Executing the code:

In [ ]:
counter = 0
df.apply(lambda row: entity_extraction.process_data(row, runnable, insert_organisations), axis=1)